In [ ]:
# Using a data which you can see that there is a yearly trend, we should really perform a train test split for
# at least a year's information. Maybe 1.5 years?

#                                              Train Test Split

In [1]:
# test_size = 18 ---> Essentially taking 1.5 years in terms of months
#                     This is provided that the data given is in months

# test_index = len(df) - test_size

# train = df.iloc[:test_index] ---> Essentially taking all the datas from beginning till before the test index as training data
# test = df.iloc[test_index:] ---> Essentially taking all the datas from test index all the way till the end

#                                             Scaling the Data

In [2]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()

# scaler.fit(train) ---> Just as before, we do not want to assume that we have any information about the future dataset.
#                        Therefore, we are only scaling our training data.

# scaled_train = scaler.transform(train)
# scaled_test = scaler.transform(test)

#                                             Time Series Generator

In [ ]:
# from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# Do note that since we are adding EarlyStopping and Validation Generator, the length of the batch MUST be lesser than the test
# set.

# length = 12 ---> Again, we are putting 12 to represent 12mths should the data provided be in months not days etc

# generator = TimeseriesGenerator(scaled_train,scaled_train, length = length, batch_size=1)
`
# X,y = generator[0]

#                                                     Model Creation

In [4]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, LSTM
# from tensorflow.keras.callbacks import EarlyStopping ---> If we wanna include Earlystopping

In [5]:
# n_features = 1

# model = Sequential()
# model.add(LSTM(100, activation = 'relu', input_shape=(length, n_features))) ---> the 100 represents the number of neurons
# [Do note that we did not include activation in the previous Sine wave example as we were using the default activation 
# function. So here we are selecting Rectified Linear Unit.]

# model.add(Dense(1))
# model.compile(optimizer = 'adam', loss='mse')

# model.summary() ---> Shows our Sequential model

# early_stop = EarlyStopping(monitor='val_loss', patience = 2) ---> Best practice to have more patience than 1 for RNN

# validation_generator = TimeseriesGenerator(scaled_test,scaled_test, length = length, batch_size=1)
#[Since we are also fitting the validation data into the training, the validation data MUST also be a generator]

# model.fit_generator(generator, epochs = 20, validation_data=validation_generator, callbacks=[early_stop])
# [The fit_generator is from the library. The generator is the variable we declared above]

#                                             Visualising the Losses

In [6]:
# losses = pd.DataFrame(model.history.history)

# losses.plot() ---> Plots the loss

#                                           Evaluate the Test Data[For-Loop]

In [7]:
# Do note that the purpose for creating this For-Loop is the predict the ALL tests results by taking the initial 12 datas from
# training dataset and then predicting the the next point after the training which is the first point of test and repeating  
# this steps whereby now we take 12(11 training and 1 predicted result) samples to predict the second point of test 
# and so on and forth.

# Step 1: Creating an empty array to store the predicted test results
# test_predictions = []

# Step 2: Taking the last 100 datasets from scaled training data as the first evaluation batch
# first_evaluation_batch = scaled_train[-length:]

# Step 3: Reshaping the batch to fit the For-Loop. We are passing in just 1 item per batch which is why we put 1 in front
# current_batch= first_evaluate_batch.reshape((1,length, n_features))

#============================================[DO NOT USE THIS CODE]==========================================================

# np.append(current_batch[:,1:,:],[[[99]]], axis=1) ---> This code is essentially saying to move the current_batch along 
#                                                        by one time step. We are using this for the for loop below.

#============================================================================================================================ 

# Step 4: Using the For-Loop to append the predicted results for test to test_predictions.
# for i in range(len(test)):

#    current_pred = model.predict(current_batch)[0]
    
#    test_predictions.append(current_pred)
    
#    current_batch = np.append(current_batch[:,1:,:],[[current_pred]], axis=1)

#                                Inversing the Scaled Data & Plotting against Test Data

In [8]:
# true_predictions = scaler.inverse_transform(test_predictions)

# test['Predictions'] = true_predictions ---> Creating another column under the test variable which was established from train
#                                             test split earlier to map the Predicted values by the model for comparison

# test.plot() ---> Plotting the Actual tests against the predicted tests.

#                                          Forecasting into the Future

In [9]:
# We are essentially retraining all of the data
# Therefore, we will be scaling all of the data

# full_scaler = MinMaxScaler()

# scaled_full_data = full_scaler.fit_transform(df) ---> Scaling all of the data

#                                            Model Creation[Forecasting]

In [10]:
# length = 12 ---> The 12 represents the 12 months. Please please don't use this for all other datas

# generator = TimeseriesGenerator(scaled_full_data, scaled_full_data, length = length, batch_size = 1)

# model = Sequential()

# model.add(LSTM(100, activation = 'relu', input_shape=(length, n_features)))
# [Do note that the RNN has been replaced with LSTM]
# [If we are satisfied with the LSTM model instead of RNN, we use LSTM]

# model.add(Dense(1))

# model.compile(optimizer='adam', loss='mse')

# model.fit_generator(generator, epochs = 8)

# [For the amount of epochs do decide, we can use the one with the lowest loss on the validation data from the graph we
# did when training the data. So in this case, the loss was the lowest in epoch 8]
# [Notice that we did not include the EarlyStop here. The reason is what is there to reference? We are forecasting leh, there
# are no validation data for use to check against since we are using the full dataset.]

#                                           Forecast into the Actual Future

In [11]:
# forecast = []

# periods = 12 ---> Do note that we did not have this in the Sine Wave Glossary. We can adjust this to be any months into the
#                   future like 12, 18, 6 months up to you. Do note that the longer you forecast, the more noise you will have
#                   as you are adding in more and more predictions from the ground truth. Best practice is to forecast the same
#                   number of periods/length as the batch length

# first_evaluate_batch = scaled_train[-length:]
# current_batch= first_evaluate_batch.reshape((1,length, n_features))

# for i in range(periods):

#    current_pred = model.predict(current_batch)[0]
    
#    forecast.append(current_pred)
    
#    current_batch = np.append(current_batch[:,1:,:],[[current_pred]], axis=1)

# [Do note that instead of putting len(test) in the for loop, we get to decide how many spaces we are 
# going to forecast into the future. We use 12 spaces in the example above]




#                                       Creating the Forecast Dataframe

In [12]:
# forecast = scaler.inverse_transform(forecast) ---> Inverting back the forecast

# forecast_index = pd.date_range(start='2019-11-01', periods=periods, freq="MS")
# start: The start is when you want the range to start at. For the example above, since the data we got is up till 2019-10-01,
#        we will start from 2019-11-01 with the predicted values.
# periods: This is the amount of periods you want to have. We put periods as it has already been established on top as 12.
#          You can manually put inside 12 also.
# freq: Its the frequency you want like in months, days etc. You can check the strings to use specifically by googling
#       'pandas frequency strings'. For this case, we have Month Start(MS) frequency.


# forecast_df = pd.DataFrame(data=forecast,index=forecast_index,columns=['Forecast'])



#                        Plotting the Forecasted Dataframe and the Dataframe itself

In [13]:
# Plotting seperately

# df.plot() ---> Original Data
# forecast_df.plot() ---> Forecasted Data


# Combining into the same plot
# ax = df.plot() ---> Do note that the ax is short form for axis used for the parameter below
# forecast_df.plot(ax=ax)
# plt.xlim('2018-01-01', '2020-12-01') ---> Zooming in on the plot